In [1]:
from typing import List
from matplotlib import pyplot as plt
import numpy as np

from importlib import reload
import pisces.models as pm

from pisces.utils import pad_to_hat, plot_scores_CDF, plot_scores_PDF, add_rocs
import matplotlib.pyplot as plt

from pisces.data_sets import DataSetObject


sets = DataSetObject.find_data_sets("../data_sets")
walch = sets['walch_et_al']
hybrid = sets['hybrid_motion']

# Walch et al. data set

## (MO) Pretrained UNet evaluation
We have the trained weights from Mads Olsen et al.'s _ResUNet_ model, saved as a tensorflow lite model with a wrapper for our system. This section calls that on accelerometer recordings in the `'walch_et_al'` data set.

In [2]:
from pisces.utils import split_analysis


mo = pm.MOResUNetPretrained(sampling_hz=32)


In [3]:
# uses all processors on your CPU to preprocess the accelerometer into spectrograms
# takes ~ 50 sec on M1 Pro (10core)
evaluations_walch, mo_preprocessed_data_walch = mo.evaluate_data_set(walch)
mo_aucs_walch = [e["auc"] for e in evaluations_walch.values()]
mo_fprs_walch = [e["roc_curve"]["fprs"] for e in evaluations_walch.values()]
mo_tprs_walch = [e["roc_curve"]["tprs"] for e in evaluations_walch.values()]


Using 16 of 16 cores (100%) for parallel preprocessing.
This can cause memory or heat issues if  is too high; if you run into problems, call prepare_set_for_training() again with max_workers = -1, going more negative if needed. (See the docstring for more info.)


getting needed X, y for 1818471
sampling hz: 32
masking
getting needed X, y for 4018081
Masked 1 PSG epochs
resampling to 32Hz (0.03125s) from 50 Hz (0.01999s)
sampling hz: 32
masking
getting needed X, y for 1449548
getting needed X, y for 2638030
getting needed X, y for 2598705
Masked 1 PSG epochs
getting needed X, y for 1455390
resampling to 32Hz (0.03125s) from 66 Hz (0.01502s)
getting needed X, y for 4426783
getting needed X, y for 46343
getting needed X, y for 1066528
sampling hz: 32
masking
getting needed X, y for 5383425
sampling hz: 32
masking
sampling hz: 32
masking
sampling hz: 32
masking
sampling hz: 32
masking
sampling hz: 32
masking
sampling hz: 32
masking
sampling hz: 32
masking
Masked 1 PSG epochs
Masked 1 PSG epochs
resampling to 32Hz (0.03125s) from 10 Hz (0.09990s)
Masked 1 PSG epochs
Masked 1 PSG epochs
Masked 1 PSG epochs
Masked 1 PSG epochs
resampling to 32Hz (0.03125s) from 50 Hz (0.01997s)
getting needed X, y for 3509524
getting needed X, y for 5498603
getting ne

In [4]:
x = np.array([[1.0, 2, 3]])
print(x)

[[1. 2. 3.]]


In [5]:
np.pad(
    x,
    pad_width=[
        (2, 1),
        (0, 3),
    ]
)

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 2., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [6]:
reload(pm)


walch_mo_run = pm.run_splits(
    pm.LeaveOneOutSplitter(),
    w=walch,
    swc_class=pm.MOResUNetPretrained,
    exclude=['7749105'],
    preprocessed_data=mo_preprocessed_data_walch
)

0it [00:00, ?it/s]

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 1.2563
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - loss: 0.7108
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - loss: 0.7057
Epoch 4/10
 1/15 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - loss: 0.5251

0it [01:21, ?it/s]


KeyboardInterrupt: 